In [1]:

from src.analysis.aggregate_trial_results import aggregate_trial_results
from src.analysis.clean_balance_filter import (
    print_rm_anova_summary,
    clean_balance_filter
)
from src.analysis.df_formatting import (
    print_nice_dataframe,
    prepare_for_repeated_measures
)
# Create summary stats using default results folder path
aggregate_trial_results = aggregate_trial_results()

# Prepare data for analysis
cleaned_df_31 = clean_balance_filter(aggregate_trial_results, 'llama_31', verbose=True)

long_df_31 = prepare_for_repeated_measures(cleaned_df_31)

# Print the updated DataFrame
print_nice_dataframe(long_df_31)

# Run the repeated measures ANOVA
# anova_results_31 = run_repeated_measures_anova(repeated_df_31)

Aggregated trial results saved to /workspaces/coordination/pipeline/4_analysis/trial_results_aggregated.csv

Data after filtering by model family and reasoning conditions:
Displaying first 20 rows (total: 189):

+------------------------------------------------------------+--------------+---------------+--------------+--------------------+------------------+----------------------+-------------------+--------------------+----------------------+-----------------------+---------------------+----------------------+----------------------+-----------------------+--------------------+------------------+---------------+-------------------+----------------+-------------------+------------------+-------------------+------------------------+-------------------+
| file_name                                                  | model_name   | temperature   | xml_prompt   | task_instruction   | task_reasoning   | task_options         | top_option_name   |   top_option_count | second_option_name   |   s

KeyError: "['control'] not in index"

In [2]:
#overview
# print_rm_anova_summary(anova_results_31)

In [ ]:
from src.analysis.visualization import (
    plot_model_comparison,
    create_comprehensive_analysis,
)
from src.analysis.df_formatting import print_nice_dataframe
# create_comprehensive_analysis(repeated_df_31)
# plot_model_comparison(repeated_df_31)
# print_nice_dataframe(repeated_df_31)